In [1]:
import pandas as pd
import numpy as np
import itertools
from datetime import date,datetime
from sklearn import preprocessing

In [2]:
holidays={
2018: {1:[1,2,3,4,5,6,7,8,13,14,20,21,27,28],2:[3,4,10,11,17,18,22,23,24,25],3:[3,4,7,8,9,10,11,17,18,24,25,31],4:[1,7,8,14,15,21,22,28,29,30],5:[1,2,5,6,8,9,12,13,19,20,26,27],6:[2,3,9,10,11,12,16,17,23,24,30],7:[1,7,8,14,15,21,22,28,29],8:[4,5,11,12,18,19,25,26],9:[1,2,8,9,15,16,22,23,29,30],10:[6,7,13,14,20,21,27,28],11:[3,4,5,10,11,17,18,24,25],12:[1,2,8,9,15,16,22,23,29,30,31]},
2019: {1:[1,2,3,4,5,6,7,8,12,13,19,20,26,27],2:[2,3,9,10,16,17,22,23,24],3:[2,3,7,8,9,10,16,17,23,24,30,31],4:[6,7,13,14,20,21,27,28,30],5:[1,2,3,4,5,8,9,10,11,12,18,19,25,26],6:[1,2,8,9,11,12,15,16,22,23,29,30],7:[6,7,13,14,20,21,27,28],8:[3,4,10,11,17,18,24,25,31],9:[1,7,8,14,15,21,22,28,29],10:[5,6,12,13,19,20,26,27],11:[2,3,4,9,10,16,17,23,24,30],12:[1,7,8,14,15,21,22,28,29,31]},
2020: {1:[1,2,3,4,5,6,7,8,11,12,18,19,25,26],2:[1,2,8,9,15,16,22,23,24,29],3:[1,7,8,9,14,15,21,22,28,29],4:[4,5,11,12,18,19,25,26,30],5:[1,2,3,8,9,10,11,16,17,23,24,30,31],6:[6,7,11,12,13,14,20,21,27,28],7:[4,5,11,12,18,19,25,26],8:[1,2,8,9,15,16,22,23,29,30],9:[5,6,12,13,19,20,26,27],10:[3,4,10,11,17,18,24,25,31],11:[1,3,4,7,8,14,15,21,22,28,29],12:[5,6,12,13,19,20,26,27,31]},
2021: {1:[1,2,3,4,5,6,7,8,9,10,16,17,23,24,30,31],2:[6,7,13,14,20,21,22,23,27,28],3:[6,7,8,13,14,20,21,27,28],4:[3,4,10,11,17,18,24,25,30],5:[1,2,3,8,9,10,15,16,22,23,29,30],6:[5,6,11,12,13,14,19,20,26,27],7:[3,4,10,11,17,18,24,25,31],8:[1,7,8,14,15,21,22,28,29],9:[4,5,11,12,18,19,25,26],10:[2,3,9,10,16,17,23,24,30,31],11:[3,4,6,7,13,14,20,21,27,28],12:[4,5,11,12,18,19,25,26,31]}
}
holidays=pd.DataFrame(list(itertools.chain.from_iterable(list(itertools.chain.from_iterable([[[date(year,month,day) for day in days] for month,days in v.items()] for year,v in holidays.items()])))),columns=['date'])
holidays.date=pd.to_datetime(holidays.date)
holidays['is_holiday']=1

In [3]:
def filetodf(filename,time_list=[]):
    # функция считывает файлы из директории и складывает данные в датафрейм
    #  входные данные: 
    #     filename - путь к файлам
    #     time_list - список полей с датой
    #  выходные данные: 
    #     pd.dataframe
    
    import glob
    import pandas as pd
    
    AllFiles = glob.glob(filename)
   
    list_ = [pd.DataFrame()]

    for file_ in AllFiles:
        columns=pd.read_csv(file_, sep=',', nrows=0).columns.str.lower().str.strip()
        df = pd.read_csv(file_,',',names=columns,parse_dates=time_list,dayfirst=True,skiprows=1)
        list_.append(df)
    
    return pd.concat(list_, axis = 0, ignore_index = True, sort=True)

In [87]:
df_sysMetrics = filetodf('input/GrpStat_OSGLOB*.dat')
df_grpMetrics = filetodf('input/GrpStat_RGALL*.dat')
df_directives = filetodf('input/GrpStat_Directives*.dat',time_list=['begin_time'])

df_sysMetrics.drop(['time'], axis=1,inplace=True)
df_grpMetrics.drop(['t_beg'], axis=1,inplace=True)

df_grpMetrics.rename(index=str, columns={"instance_number": "host", "usr_group": "consumer_group"}, inplace=True)
df_directives.rename(index=str, columns={"instance_number": "host", "begin_time": "time"}, inplace=True)

df_grpMetrics['consumer_group']=df_grpMetrics['consumer_group'].str.strip().str.lower()
df_directives['consumer_group']=df_directives['consumer_group'].str.strip().str.lower()
df_directives['plan_name']=df_directives['plan_name'].str.strip().str.lower()

df_grpMetrics.replace(' None', np.NaN, inplace=True)
df_directives.replace(' None', np.NaN, inplace=True)
df_sysMetrics.replace(' None', np.NaN, inplace=True)

df_directives.parallel_target_percentage=df_directives.parallel_target_percentage.astype(float)
df_directives.parallel_degree_limit_p1=df_directives.parallel_degree_limit_p1.astype(float)
df_directives.max_utilization_limit=df_directives.max_utilization_limit.astype(float)
df_grpMetrics.pga=df_grpMetrics.pga.astype(float)

In [88]:
#джойним все данные до уровня host, snap_id, consumer_group
df=df_grpMetrics.set_index(['host','snap_id','consumer_group']) \
.join(df_directives.set_index(['host','snap_id','consumer_group']))\
.reset_index()\
.merge(df_sysMetrics,on=['host','snap_id'])

df = df[df.consumer_group!='other_groups']
df = df[df.consumer_group!='ods2exa_group']

# ресэмплинг до часа, чтобы сгладить ряд
# заполняем отсутствующие значения средними
# фильтруем неполные дни в начале и в конце

df = df.sort_values(['host','consumer_group','time'])
df = df.set_index(pd.DatetimeIndex(df['time']))
#-----------------------------
#df.loc[(df.consumer_group=='adw_unload_group')&(df.plan_name=='sandbox_plan'),'mgmt_p1']=26.
#-----------------------------

df.max_utilization_limit.fillna(100,inplace=True)
df.parallel_target_percentage.fillna(100,inplace=True)
df.parallel_degree_limit_p1.fillna(100,inplace=True)

#df.loc[:,'cpu_pct_maxul']=df.cpu/df.max_utilization_limit
#df.loc[:,'px_pct_parallel']=df.pxqry/(df.parallel_target_percentage*1400.0/100.0)
#df.loc[df.px_pct_parallel>2,'px_pct_parallel']=2.0
#df.loc[df.cpu_pct_maxul>2,'cpu_pct_maxul']=2.0
#df['CPU_min_treshold']=(df.mgmt_p1/df.max_utilization_limit)

columns=df.columns.drop(['host','consumer_group','time','snap_id','plan_name'],errors='ignore')

df=df.groupby(['host','consumer_group'])[columns].apply(lambda df: df.resample('H').mean().fillna(df.mean(axis=0))).reset_index(['host','consumer_group'])
#df=df.groupby(['host','consumer_group'])[columns].apply(lambda df: df.resample('H').mean()).reset_index().dropna()
df=df.groupby(['host','consumer_group',df.index.date]).filter(lambda x: (x.index.min().hour==0)&(x.index.max().hour==23)).reset_index()


In [ ]:
#null_columns=df.columns[df.isnull().any()]
#df[null_columns].isnull().sum()

In [89]:
# нормируем признаки 
host_cons=[(host,group) for host,group in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values]
scaler = preprocessing.MinMaxScaler()
#scaler = preprocessing.StandardScaler()
df_scaled=pd.DataFrame()
for (host,consumer_group) in host_cons:
    df_sample=df[(df.host==host)&(df.consumer_group==consumer_group)].reset_index(drop=True)
    df_sample=pd.concat([df_sample.drop(columns=columns),pd.DataFrame(scaler.fit_transform(df_sample[columns]),columns=columns)],axis=1,sort=False)
    df_scaled=pd.concat([df_scaled,df_sample],sort=False)



In [90]:

# datetime features

    #day_of_month
    #day_of_week
    #month_of_year
    #hour_of_day
    #is_holiday
    #is_weekend
    #quarter_of_year


df_scaled.loc[:,('day_of_months')]=df_scaled.time.dt.day
df_scaled.loc[:,('day_of_week')]=df_scaled.time.dt.dayofweek
df_scaled.loc[:,('month_of_year')]=df_scaled.time.dt.month
df_scaled.loc[:,('hour_of_day')]=df_scaled.time.dt.hour
df_scaled.loc[:,('is_weekend')]=df_scaled.time.dt.dayofweek.isin([5,6])*1
df_scaled.loc[:,('quarter_of_year')]=df_scaled.time.dt.quarter

df_scaled['date']=pd.to_datetime(df_scaled.time.dt.strftime('%Y-%m-%d'))
df_scaled=df_scaled.merge(holidays,on='date', how='left').drop(columns=['date'])
df_scaled.loc[:,('is_holiday')].fillna(0,inplace=True)

#dummy features

#dummy_columns=['day_of_months','day_of_week','month_of_year','hour_of_day','quarter_of_year','host','consumer_group']
dummy_columns=['day_of_months','day_of_week','month_of_year','hour_of_day','quarter_of_year']
df_scaled=pd.concat(
    [
        df_scaled,
        pd.get_dummies(
            df_scaled[dummy_columns].astype(str),
            prefix=dummy_columns)
    ],
    axis=1,
    sort=False
)

In [ ]:
# добавляем лаги исходного ряда в качестве признаков


In [ ]:
#df_week_shift=df[columns].shift(24*7)
#df_day_shift=df[columns].shift(24)
#for (host,group) in df.drop_duplicates(['host','consumer_group'])[['host','consumer_group']].values:
#    t=df[(df.host==host)&(df.consumer_group==group)][columns]
#    for row in t.index:
#        if t.loc[row].isnull().all():
#            row_day=df_day_shift.loc[row,columns]
#            row_week=df_week_shift.loc[row,columns]
#            if row_week.isnull().all():
#                if row_day.isnull().all():
#                    df.loc[row,columns]=t.loc[:,columns].mean(axis=0)
#                else:
#                    df.loc[row,columns]=row_day
#            else:
#                df.loc[row,columns]=row_week
#df.fillna(0,inplace=True)
            
            

In [91]:
df_scaled.to_csv('clear_data/rm_features.csv',';',index=False)

In [92]:
df_scaled

,time,host,consumer_group,cpu,dbtime,iowait,oth_wait,pga,pxenq,pxqry,...,hour_of_day_22,hour_of_day_23,hour_of_day_3,hour_of_day_4,hour_of_day_5,hour_of_day_6,hour_of_day_7,hour_of_day_8,hour_of_day_9,quarter_of_year_4
0,2018-11-02 00:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000023,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
1,2018-11-02 01:00:00,1,adw_unload_group,0.083485,0.142540,0.043084,0.280353,0.034612,0.028169,0.046452,...,0,0,0,0,0,0,0,0,0,1
2,2018-11-02 02:00:00,1,adw_unload_group,0.432849,0.331669,0.040816,0.187638,0.395480,0.000000,0.100645,...,0,0,0,0,0,0,0,0,0,1
3,2018-11-02 03:00:00,1,adw_unload_group,0.292196,0.236273,0.041950,0.147903,0.452624,0.239437,0.077419,...,0,0,1,0,0,0,0,0,0,1
4,2018-11-02 04:00:00,1,adw_unload_group,0.000907,0.001109,0.000000,0.002208,0.007798,0.000000,0.001290,...,0,0,0,1,0,0,0,0,0,1
5,2018-11-02 05:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000106,0.000000,0.000000,...,0,0,0,0,1,0,0,0,0,1
6,2018-11-02 06:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000106,0.000000,0.000000,...,0,0,0,0,0,1,0,0,0,1
7,2018-11-02 07:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000121,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,1
8,2018-11-02 08:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000155,0.000000,0.000000,...,0,0,0,0,0,0,0,1,0,1
9,2018-11-02 09:00:00,1,adw_unload_group,0.000000,0.000000,0.000000,0.000000,0.000093,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,1
